In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
# import tensorflow as tf
import random

In [17]:
num_classes = 7

# --- Custom loss/metrics definitions (not used but needed for loading model) ---
def dice_coef(y_true, y_pred):
    total_dice = 0.0
    num_class = 0.0
    for class_idx in range(num_classes):
        y_true_class = y_true[..., class_idx]
        y_pred_class = y_pred[..., class_idx]
        y_true_f = tf.keras.backend.flatten(y_true_class)
        y_pred_f = tf.keras.backend.flatten(y_pred_class)
        intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
        intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
        dice = (2. * intersection) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1e-7)
        total_dice = total_dice + dice
        num_class = num_class + 1.0
    mean_dice_score = total_dice / num_class
    return mean_dice_score

def tpr(y_true, y_pred, threshold=0.5):
    total_tpr = 0
    num_class = 0
    for class_idx in range(num_classes):
        y_true_class = y_true[..., class_idx]
        y_pred_class = y_pred[..., class_idx]
        y_pred_pos = tf.cast(y_pred_class > threshold, tf.float32)
        y_true_pos = tf.cast(y_true_class > threshold, tf.float32)
        true_pos = tf.reduce_sum(tf.cast(tf.logical_and(y_true_pos == 1, y_pred_pos == 1), tf.float32))
        actual_pos = tf.reduce_sum(tf.cast(y_true_pos, tf.float32))
        tpr = true_pos / (actual_pos + tf.keras.backend.epsilon())
        total_tpr += tpr
        num_class += 1
    mean_tpr = total_tpr / num_class
    return mean_tpr

def fpr(y_true, y_pred, threshold=0.5):
    total_fpr = 0
    num_class = 0
    for class_idx in range(num_classes):
        y_true_class = y_true[..., class_idx]
        y_pred_class = y_pred[..., class_idx]
        y_pred_pos = tf.cast(y_pred_class > threshold, tf.float32)
        y_true_neg = tf.cast(y_true_class <= threshold, tf.float32)
        false_pos = tf.reduce_sum(tf.cast(tf.logical_and(y_true_neg == 1, y_pred_pos == 1), tf.float32))
        actual_neg = tf.reduce_sum(tf.cast(y_true_neg, tf.float32))
        fpr = false_pos / (actual_neg + tf.keras.backend.epsilon())
        total_fpr += fpr
        num_class += 1
    mean_fpr = total_fpr / num_class
    return mean_fpr

def combined_loss(y_true, y_pred):
    cce_loss = tf.keras.losses.CategoricalCrossentropy()(y_true, y_pred)
    dice_loss = 1 - dice_coef(y_true, y_pred)
    return cce_loss + dice_loss

# --- Preprocessing ---
def preprocess_slice(slice_img):
    if np.max(slice_img) > 0:
        slice_norm = slice_img / np.max(slice_img)
    else:
        slice_norm = slice_img
    return np.expand_dims(slice_norm, axis=-1)

# --- Prediction on 3D image ---
def predict_3d(model, nii_path, save_png_slices=False, ground_truth_path=None):
    nii_img = nib.load(nii_path)
    img_data = nii_img.get_fdata()
    pred_volume = np.zeros_like(img_data)

    if ground_truth_path and os.path.exists(ground_truth_path):
        gt_data = nib.load(ground_truth_path).get_fdata()
    else:
        gt_data = np.zeros_like(img_data)

    randSlice = random.randint(0, img_data.shape[2])
    for z in range(img_data.shape[2]):
        slice_img = img_data[:, :, z]
        input_slice = np.expand_dims(preprocess_slice(slice_img), axis=0)
        prediction = model.predict(input_slice, verbose=0)
        predicted_label = np.argmax(prediction, axis=-1)
        pred_volume[:, :, z] = predicted_label[0]

        if save_png_slices and z==randSlice:
            save_png_for_slice(slice_img, gt_data[:, :, randSlice], predicted_label[0], randSlice)

    # for z in range(img_data.shape[2]):
    #     slice_img = img_data[:, :, z]
    #     input_slice = np.expand_dims(preprocess_slice(slice_img), axis=0)
    #     prediction = model.predict(input_slice, verbose=0)
    #     predicted_label = np.argmax(prediction, axis=-1)
    #     pred_volume[:, :, z] = predicted_label[0]

    #     if save_png_slices:
    #         save_png_for_slice(slice_img, gt_data[:, :, z], predicted_label[0], z)

    return pred_volume

# --- PNG saving ---
def save_png_for_slice(raw_slice, ground_truth, predicted_label, z):
    plt.figure(figsize=(16, 8))
    
    plt.subplot(131)
    plt.title('Raw Slice')
    plt.imshow(raw_slice, cmap='gray')
    plt.axis('off')

    plt.subplot(132)
    plt.title('Ground Truth')
    plt.imshow(ground_truth, cmap='jet', vmin=0, vmax=6)
    plt.axis('off')

    plt.subplot(133)
    plt.title('Prediction')
    plt.imshow(predicted_label, cmap='jet', vmin=0, vmax=6)
    plt.axis('off')

    plt.show()
    plt.close()

# --- Save as NIfTI ---
def save_prediction_as_nii(pred_volume, original_nii_path, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    orig_nii = nib.load(original_nii_path)
    pred_nii = nib.Nifti1Image(pred_volume, affine=orig_nii.affine, header=orig_nii.header)
    basename = os.path.basename(original_nii_path)
    nib.save(pred_nii, os.path.join(save_dir, basename))
    print(f"Saved prediction: {basename}")

# --- Main batch processing ---
def process_all_volumes(model_path, image_folder, gt_folder, save_nii_folder):
    model = tf.keras.models.load_model(model_path, custom_objects={
        'dice_coef': dice_coef, 'tpr': tpr, 'fpr': fpr, 'combined_loss': combined_loss
    })

    nii_files = [f for f in os.listdir(image_folder) if f.endswith('.nii')]
    for i, fname in enumerate(sorted(nii_files)):
        image_path = os.path.join(image_folder, fname)
        gt_path = os.path.join(gt_folder, fname)  # Assuming same filename

        print(f"Processing {fname}...")

        save_pngs = (i%5==0)
        pred_vol = predict_3d(
            model,
            image_path,
            save_png_slices=save_pngs,
            ground_truth_path=gt_path
        )
        save_prediction_as_nii(pred_vol, image_path, save_nii_folder)

# --- Example usage ---
# model_path = 'C:/Users/Mittal/Desktop/thoracic_seg/models/multi_thoracic_unet_model_0.h5'
# image_folder = 'C:/Users/Mittal/Desktop/thoracic_seg/raw_images/'
# gt_folder = 'C:/Users/Mittal/Desktop/thoracic_seg/segmentations/'
# save_nii_folder = 'C:/Users/Mittal/Desktop/thoracic_seg/unet_niipredictions/'

model_path = "C:\\Users\\Satvi\\OneDrive\\Desktop\\thoracic_seg\\models\\multi_thoracic_unet_model_0.h5"
image_folder = "C:\\Users\\Satvi\\OneDrive\\Desktop\\thoracic_seg\\raw_images\\"
gt_folder = 'C:\\Users\\Satvi\\OneDrive\\Desktop\\thoracic_seg\\segmentations\\'
save_nii_folder = 'C:\\Users\\Satvi\\OneDrive\\Desktop\\thoracic_seg\\unet_niipredictions\\'

process_all_volumes(model_path, image_folder, gt_folder, save_nii_folder)

TypeError: Error when deserializing class 'Conv2DTranspose' using config={'name': 'conv2d_transpose', 'trainable': True, 'dtype': 'float32', 'filters': 512, 'kernel_size': [2, 2], 'strides': [2, 2], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None, 'output_padding': None}.

Exception encountered: Unrecognized keyword arguments passed to Conv2DTranspose: {'groups': 1}

In [2]:
# raw_image_directory = 'C:/Users/Mittal/Desktop/thoracic_seg/raw_images/'
# ground_truth_directory = 'C:/Users/Mittal/Desktop/thoracic_seg/segmentations/'
# unet_directory = 'C:/Users/Mittal/Desktop/thoracic_seg/unet_niipredictions/'
# unetpp_directory = 'C:/Users/Mittal/Desktop/thoracic_seg/unet++_predictions/'

raw_image_directory = "C:\\Users\\Satvi\\OneDrive\\Desktop\\thoracic_seg\\raw_images\\"
ground_truth_directory = 'C:\\Users\\Satvi\\OneDrive\\Desktop\\thoracic_seg\\segmentations\\'
unet_directory = 'C:\\Users\\Satvi\\OneDrive\\Desktop\\thoracic_seg\\unet_oldniipredictions\\'

raw_images = sorted(os.listdir(raw_image_directory))
segmentations = sorted(os.listdir(ground_truth_directory))
unet_predictions = sorted(os.listdir(unet_directory))
# unetpp_predictions = sorted(os.listdir(unetpp_directory))

image_dataset = []
mask_dataset = []
unet_prediction_dataset = []
# unetpp_prediction_dataset = []
image_names = []

for image_name in raw_images:    
    if (image_name.split('.')[1] == 'nii'):
        base_name = image_name.split('.')[0]
        image = nib.load(raw_image_directory+image_name).get_fdata()
        segmentation = nib.load(ground_truth_directory+image_name).get_fdata()
        unet = nib.load(unet_directory+image_name).get_fdata()
        # unetpp = nib.load(unetpp_directory+image_name).get_fdata()
        
        image_dataset.append(np.array(image))
        mask_dataset.append(np.array(segmentation))
        unet_prediction_dataset.append(np.array(unet))
        # unetpp_prediction_dataset.append(np.array(unetpp))
        image_names.append(base_name)

In [4]:
num_classes = 7
stats = {}

# allComparisons = {"GroundTruth": mask_dataset, "Unet": unet_prediction_dataset, "Unet++":unetpp_prediction_dataset}
allComparisons = {"GroundTruth": mask_dataset, "Unet": unet_prediction_dataset}
slices = {"Mid-Slice+1":1, "Mid-Slice":0, "Mid-Slice-1":-1}

for i in range(len(image_dataset)):
    image = image_dataset[i]
    image_name = image_names[i]
    mid_slice = image.shape[2]//2
    for slice_label, offset in slices.items():
        row_key = f"{image_name}_{slice_label}"
        stats[row_key] = {}
        for model_name, model_masks in allComparisons.items():
            for j in range(num_classes-2):
                mask_class = model_masks[i]==(j+2)
                slice_data = (mask_class*image)[:,:,mid_slice+offset] 
                slice_data = slice_data[slice_data!=0]
                mean = np.mean(slice_data) if slice_data.size>0 else 0
                std = np.std(slice_data) if slice_data.size>0 else 0
                stats[row_key][model_name, f"Cylinder {j+1}", "Mean"] = mean
                stats[row_key][model_name, f"Cylinder {j+1}", "Std"] = std

In [14]:
import pandas as pd

df = pd.DataFrame.from_dict(stats, orient='index')
df.columns = pd.MultiIndex.from_tuples(df.columns, names=['Model', 'Cylinder', 'Stat'])
df.index.name = "Image_Slice"
df

Model                                       GroundTruth                       \
Cylinder                                     Cylinder 1           Cylinder 2   
Stat                                               Mean       Std       Mean   
Image_Slice                                                                    
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice+1       41.316129  5.242204  33.524590   
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice         40.333333  7.166816  31.925170   
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice-1       41.094937  8.441750  33.095890   
A21_02_FatFrac_2D_FAM_BH_18_Mid-Slice+1        0.000000  0.000000   0.000000   
A21_02_FatFrac_2D_FAM_BH_18_Mid-Slice          0.000000  0.000000   0.000000   
...                                                 ...       ...        ...   
A21_19_FatFrac_IDEAL_IQ_PDFF_22_Mid-Slice      0.000000  0.000000   0.000000   
A21_19_FatFrac_IDEAL_IQ_PDFF_22_Mid-Slice-1    0.000000  0.000000   0.000000   
A21_31_FatFrac_3D_Iron_Quant_17_Mid-Slice+1   41.055556  9.830822  31.108108   
A21_31_FatFrac_3D_Iron_Quant_17_Mid-Slice     41.479730  6.489565  30.439189   
A21_31_FatFrac_3D_Iron_Quant_17_Mid-Slice-1   41.162162  6.650598  30.560811   

Model                                                                        \
Cylinder                                               Cylinder 3             
Stat                                               Std       Mean       Std   
Image_Slice                                                                   
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice+1       5.315016  25.000000  4.876862   
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice         4.796666  21.632653  4.552164   
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice-1       5.916460  22.713287  3.988835   
A21_02_FatFrac_2D_FAM_BH_18_Mid-Slice+1       0.000000   0.000000  0.000000   
A21_02_FatFrac_2D_FAM_BH_18_Mid-Slice         0.000000   0.000000  0.000000   
...                                                ...        ...       ...   
A21_19_FatFrac_IDEAL_IQ_PDFF_22_Mid-Slice     0.000000   0.000000  0.000000   
A21_19_FatFrac_IDEAL_IQ_PDFF_22_Mid-Slice-1   0.000000   0.000000  0.000000   
A21_31_FatFrac_3D_Iron_Quant_17_Mid-Slice+1  10.306214  20.858108  4.753392   
A21_31_FatFrac_3D_Iron_Quant_17_Mid-Slice     9.044355  21.006757  4.855335   
A21_31_FatFrac_3D_Iron_Quant_17_Mid-Slice-1   7.745292  20.216216  5.035738   

Model                                                                        \
Cylinder                                    Cylinder 4           Cylinder 5   
Stat                                              Mean       Std       Mean   
Image_Slice                                                                   
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice+1      16.512821  7.062136   9.156522   
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice        14.671429  2.844651   3.573643   
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice-1      12.957447  3.956989   0.623656   
A21_02_FatFrac_2D_FAM_BH_18_Mid-Slice+1       0.000000  0.000000   0.000000   
A21_02_FatFrac_2D_FAM_BH_18_Mid-Slice         0.000000  0.000000   0.000000   
...                                                ...       ...        ...   
A21_19_FatFrac_IDEAL_IQ_PDFF_22_Mid-Slice     0.000000  0.000000   0.000000   
A21_19_FatFrac_IDEAL_IQ_PDFF_22_Mid-Slice-1   0.000000  0.000000   0.000000   
A21_31_FatFrac_3D_Iron_Quant_17_Mid-Slice+1  11.436364  7.655690  -1.149533   
A21_31_FatFrac_3D_Iron_Quant_17_Mid-Slice    11.830303  7.445661  -0.262500   
A21_31_FatFrac_3D_Iron_Quant_17_Mid-Slice-1  12.347305  7.694720  -0.384615   

Model                                                       Unet      \
Cylinder                                              Cylinder 1       
Stat                                              Std       Mean Std   
Image_Slice                                                            
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice+1      4.713701          0   0   
A21_02_FatFrac_2D_FAM_BH_14_Mid-Slice        1.674131          0   

In [ ]:
# Save DataFrame to an Excel file
df.to_excel('stats_dataframe.xlsx', index=True)  # Include index (file names)
